In [1]:
import transformers
import torch
import pandas as pd
import json
import os

In [2]:
from huggingface_hub import login
login('login_key')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda", 
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [4]:
input1_text_user="Can you provide a summary of any recalls related to disc brake failures for the Toyota Camry?"

In [5]:
messages = [
        {"role": "system", "content": "You are only a python dictionary creator don't do anything else. Take input which is about vehicle given by the user and convert only into the following format:  make: capitalize(FORD or TOYOTA else empty), model: if any captalize(content), year: if any given else empty, issue: focus this part only keywords related to vehicle part  which is given in input don't add anything by yourself "}, 
        {"role": "user", "content": input1_text_user}, 
    ]

    # Generate the summary using the pipeline
outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    
    # Extract individual summaries from the generated text
generated_text = outputs[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [6]:
processed_text=generated_text[-1]['content']
processed_text

"{'make': 'TOYOTA','model': 'CAMRY', 'year': '', 'issue': 'disc brake failures'}"

In [7]:

import ast
vehicle_info = ast.literal_eval(processed_text)


query = {
    "make": "",
    "model": "",
    "year": "",
    "issue": "",
}

query.update(vehicle_info)

In [27]:
query

{'make': 'TOYOTA',
 'model': 'CAMRY',
 'year': '',
 'issue': 'disc brake failures'}

In [8]:
messages = [
        {"role": "system", "content": "You are a python set creator don't do anything else : just take the text given in input and make set of keywords only given in text related to vehicle mechanical component  only not failures"}, 
        {"role": "user", "content": query['issue']}, 
    ]

outputs = pipeline(
        messages,
        max_new_tokens=256,
    )

keyword_text = outputs[0]["generated_text"][-1]['content']

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [9]:
keyword_text 

"{'disc', 'brake'}"

In [10]:
original_set = eval(keyword_text) 

keywords_vehicle_part = {word.capitalize() for word in original_set}.union({word.upper() for word in original_set}).union(original_set)

print(keywords_vehicle_part)

{'BRAKE', 'Brake', 'Disc', 'disc', 'DISC', 'brake'}


In [11]:
with open("/kaggle/input/data-predii-result/database_chatbox.json", "r") as file:
    data = json.load(file)

In [12]:
def filter_records(data, query, keywords):
    filtered_results = []
    
    for record in data:
        # Match make and model
        if record['make'] == query['make'] :
            if query['model'] and record['model'] != query['model']:
                continue
            
            # Match year if specified
            if query['year'] and record['year'] != query['year']:
                continue
            
            # Check if any keyword is in the defect_summary
            if any(keyword in record['defect_summary'] for keyword in keywords):
              filtered_results.append(record)
    
    return filtered_results

# Get filtered records
records = filter_records(data, query, keywords_vehicle_part)

In [13]:
len(records)

1

In [14]:
result_string = f"make: {query['make']}\n"
if query['model']:
    result_string += f"model: {query['model']}\n"
if query['year']:
    result_string += f"year: {query['year']}\n"
# Iterate over each filtered record
for record in records:
    # Append model if it wasn't specified in the query
    if not query['model']:
        result_string += f"model: {record['model']}  "
    
    # Append year if it wasn't specified in the query
    if not query['year']:
        result_string += f"year: {record['year']}  "
    
    # Append merged summary, escaping quotes
    escaped_summary = record['merged_summary'].replace('"', '\\"')
    result_string += f"merged_summary: {escaped_summary}\n\n"

In [15]:
result_string

'make: TOYOTA\nmodel: CAMRY\nyear: 1997  merged_summary: A condition can occur in vehicles used in extremely cold climates where accumulated moisture in the brake vacuum hose can temporarily freeze, leading to a loss of vacuum assist to the brakes, which can increase stopping distances and potentially cause a vehicle crash. Dealers will install a new design brake vacuum hose to resolve the issue.\n\n'

In [17]:
input="You are a summarizer."+" "+input1_text_user

In [18]:
input

'You are a summarizer. Can you provide a summary of any recalls related to disc brake failures for the Toyota Camry?'

In [19]:
messages = [
        {"role": "system", "content": input}, 
        {"role": "user", "content": result_string}, 
    ]

outputs = pipeline(
        messages,
        max_new_tokens=256,
    )

output_to_show_as_result = outputs[0]["generated_text"][-1]['content']

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [20]:
output_to_show_as_result

'Based on the given information, I was unable to find any information about a recall specifically for the 1997 Toyota Camry regarding disc brake failures. However, I did find a recall related to the brake system. \n\nRecall Number: 97V131000\n\nSummary: A condition can occur in vehicles used in extremely cold climates where accumulated moisture in the brake vacuum hose can temporarily freeze, leading to a loss of vacuum assist to the brakes, which can increase stopping distances and potentially cause a vehicle crash. Dealers will install a new design brake vacuum hose to resolve the issue.'

In [21]:
output_directory="/kaggle/working/output.json"

In [22]:
update_row={
    'input': input1_text_user,
    'output': output_to_show_as_result
}

In [23]:
if os.path.exists(output_directory):
    with open(output_directory, "r") as file:
        all_data = json.load(file)
else:
    all_data = []

# Append the new result to the data
all_data.append(update_row)

# Write the updated data back to the JSON file
with open(output_directory, "w") as file:
    json.dump(all_data, file, indent=4)